In [1]:
import os
import json
from tqdm import tqdm_notebook
import time
import numpy as np
import pandas as pd
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
matplotlib.rcParams['figure.figsize'] = (8, 8)
sns.set_style('whitegrid')

### Задание 1. KNN

Все дальнейшие эксперименты задание №1 предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [ ]:
# Считываем данные
data = pd.read_csv('.../amazon.csv')
data.head()

In [ ]:
data.shape

In [ ]:
# доля положительных примеров
data.ACTION.mean()

In [ ]:
# число значений у признаков
for col_name in data.columns:
    print col_name, len(data[col_name].unique())

In [ ]:
# НЕ МЕНЯЕМ RANDOM_STATE!!!
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

#### 1.1 [2 балла]

Реализуйте три функции расстояния на категориальных признаках, которые описаны [вот здесь](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf).

Проще всего будет определить метрики как [user-defined distance](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

In [43]:
def overlap(x, z):
    # Ваш код здесь
    pass

def flattened_overlap(x, z):
    # Ваш код здесь
    pass

def log_overlap(x, z):
    # Ваш код здесь
    pass

#### 1.2 [1 балл]

Подсчитайте для каждой из метрик качество на тестовой выборке X_test при числе соседей k=10. Мера качества — AUC-ROC.
Какая функция расстояния оказалась лучшей?

In [44]:
# Ваш код здесь:


#### 1.3 [2 балла]

Подберите лучшее (на тестовой выборке) число соседей k для каждой из функций расстояния. Какое наилучшее качество удалось получить?
Для подбора можно использовать любые средства из sklearn.

In [45]:
# Ваш код здесь:


#### 1.4 [6 баллов]

Реализуйте [счётчики](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf) для кодирование категориальных признаков.

А именно, каждый категориальный признак нужно заменить на три:

<ul>
<li>Число counts объектов в обучающей выборке с таким же значением признака.</li>

<li>Число successes объектов первого класса (y=1) в обучающей выборке с таким же значением признака.</li>

<li>Сглаженное отношение двух предыдущих величин: (successes + 1) / (counts + 2).</li>
</ul>

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать фолдинг: разбить обучающую выборку на n частей, и для i-й части считать counts и successes по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать n=3.

Посчитайте на тесте AUC-ROC метода k ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей k.

In [46]:
# Ваш код здесь:


#### 1.5 [3 балла]

Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i,f_j), \ i<j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода k ближайших соседей с наилучшим k (с фолдингом и без).

In [47]:
# Ваш код здесь:


### Задание  2. Решающее дерево

#### 2.1 [10 баллов]

Реализуйте свой класс Решающее дерево (для задачи классификации и регрессии). Шаблон класса представлен ниже, однако дерево не обязательно реализовывать именно по такому шаблону, главное, чтобы у ваше класса был метод .fit() - по двумерной матрице объект-признак и одномерному вектору таргетов получает оптимальные предикаты и метод .predict() - по двумерной матрице объект-признак возвращает одномерный вектор предсказаний.

Построение дерева должно осуществляться согласно базовому жадному алгоритму. Выбор лучшего разбиения необходимо производить по критерию, поданному в качестве аргумента в init ("Gini" или "Entropy" - для задачи классификации и "Variance" - для задачи регрессии). Критерий останова: все объекты в листе относятся к одному классу. Ответ в листе: класс объектов, находящихся в нем. Для категориальных признаков необходимо выполнить преобразование, описанное на [семинаре](https://github.com/esokolov/ml-course-msu/blob/master/ML16/lecture-notes/Sem04_trees.pdf) в разделе "Учет категориальных признаков".

In [ ]:
class DecisionTree:
    def __init__(self, criterion):
        """
        criterion: Какой критерий расщепления использовать: "Gini" или "Entropy" в случае 
        задачи классификации и "Variance" - в случае задачи регрессии 
        feature_numbers_and_splits - list список, содержащий оптимальные номера признаков и 
        разбиения. Пример: 
                [(0, 0.34), 
                           [(1, -1.16), 
                                       [(0, 13.53), 
                                                   [(), 
                                                    (0, 2.0), 
                                                           [(), 
                                                            ()]], 
                                        ()
                                                        ], 
                            (4, -0.33), 
                                       [(),  
                                        (3, 11.2), 
                                                   [(), 
                                                    ()]]]]
        Этот список символизирует следующее дерево:
                                    
                                    [x_0 < 0.34]
                                    /          \
                        [x_1 < -1.16]          [x_4 < -0.33]
                        /          \            /         \
                [x_0 < 13.53]     Лист       Лист       [x_3 < 11.2]
                   /     \                               /        \
                 Лист   [x_0 < 2.0]                    Лист      Лист
                          /     \ 
                        Лист    Лист
        """
        self.criterion = criterion
        self.feature_numbers_and_splits = []
        # Здесь могут быть ещё какие-то параметры
        
    def criterion_count(self, node_y):
        """
        node_y: np.array размера (|R|) - вектор таргетов, для объектов в этом листе.
        
        return:
            node_criterion - float значения критерия ('Gini', 'Entropy' или 'Variance') для 
            листа, содержащего объекты с таргетами node_y
        """
        # Ваш код здесь
        
        return node_critetion
    
    def _find_best_split(self, node_X, node_y):
        """
        node_X: np.array размера (|R| x d) - матрица объект-признак. |R| - число объектов в 
        этом листе, d - число признаков. 
        node_y: np.array размера (|R|) - вектор таргетов, для объектов в этом листе.
        
        return: 
            best_feature_number - int номер признака от 0 до d-1 включительно, 
            дающего лучшее разбиение
            best_split_value - float значения предиката для деления, которое обеспечивает
            наибольшее значение критерия ('Gini', 'Entropy', 'Variance')
        """
        # Ваш код здесь
        
        return best_feature_number, best_split_value

        
    def _split_leaf(self, node_X, node_y, feature_number, split_value):
        """
        node_X: np.array размера (|R| x d) - матрица объект-признак. |R| - число объектов в 
        этом листе, d - число признаков. 
        node_y: np.array размера (|R|) - вектор таргетов, для объектов в этом листе.
        feature_number: int номер признака от 0 до d-1 включительно, по которому произвести
        разбиение листа на два
        split_value: float трешхолд, по которому производить разбиение
        Иными словами, предикат для разбиения выглядит так: [feature_number < split_value]
        
        return: 
        Возвращаем 4 объекта: 
                left_node_X (np.array размера (|Rl| x d)), 
                left_node_y (np.array размера (|Rl|)), 
                right_node_X (np.array размера (|Rr| x d)), 
                right_node_y (np.array размера (|Rr|))
        """
        # Ваш код здесь
        
        return left_node_X, left_node_y, right_node_X, right_node_y

    
    def fit(self, X, y, cat_features, verbose=False):
        """
        X - np.array размера (n x d) - матрица объект-признак для обучающей выборки, на которой
        мы хотим обучить дерево
        y - np.array размера (n) - вектор таргетов для обучающей выборки, на которой
        мы хотим обучить дерево
        verbose - (True/False) нужно ли выводить итерации обучения.
        cat_features - np.array размера (d) - вектор из 0 и 1, символизирующий какие признаки
        являются вещественными, а какие категориальными. (0 - признак вещественный, 1 - кате-
        гориальный)
        
        return:
            метод ничего не возвращает, но заполняет self.feature_numbers_and_splits, как 
            показано в init'e
        """
        # Ваш код здесь
        
    def predict(self, X, cat_features):
        """
        X - np.array размера (m x d) - матрица объект-признак для выборки, на которой хотим
        получить предсказания.
        cat_features - np.array размера (d) - вектор из 0 и 1, символизирующий какие признаки
        являются вещественными, а какие категориальными. (0 - признак вещественный, 1 - кате-
        гориальный)
        
        return:
            y_pred - np.array размера (m) - вектор предсказаний для матрицы X.
        """
        # Ваш код здесь
        
        return y_pred
        
        
        

#### 2.2 [3 балла]

Протестируйте свое решающее дерево на датасете [mushrooms](https://archive.ics.uci.edu/ml/datasets/Mushroom). Вам нужно скачать таблицу agaricus-lepiota.data (из [Data Folder](https://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/)), прочитать ее с помощью pandas, применить к каждому столбцу LabelEncoder (из sklearn), чтобы преобразовать строковые имена категорий в натуральные числа. Первый столбец - это целевая переменная (e-edible, p-poisonous) Мы будем измерять качество с помощью accuracy, так что нам не очень важно, что будет классом 1, а что - классом 0. Обучите решающее дерево на половине случайно выбранных объектов (признаки в датасете категориальные) и сделайте предсказания для оставшейся половины. Вычислите accuracy.

У вас должно получиться значение accuracy, равное единице (или очень близкое к единице), и не очень глубокое дерево.

In [29]:
# Ваш код здесь:


#### 2.3 [6 баллов]

Загрузите следующие наборы данных (напомним, что pandas умеет загружать файлы по url, в нашем случае это файл *.data), предварительно ознакомившись с описанием признаков и целевой переменной в каждом из них (она записаны в Data Folder, в файле *.names):

<ul>
    
<li> 

[mushrooms](https://archive.ics.uci.edu/ml/datasets/Mushroom) (загрузили в предыдущем пункте, классы записаны в нулевом столбце) 
</li>

<li> 

[tic-rac-toe](https://archive.ics.uci.edu/ml/datasets/Tic-Tac-Toe+Endgame) (классы записаны в последнем столбце) 
</li>

<li>
    
[cars](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation) (классы записаны в последнем столбце, считаем что unacc, acc - это класс 0, good, vgood - класс 1)
</li>

<li>
    
[nurcery](https://archive.ics.uci.edu/ml/datasets/Nursery) (классы записаны в последнем столбце, считаем, что not_recom и recom - класс 0, very_recom, priority, spec_prior - класс 1)
</li>
</ul>

Закодируйте категориальные признаки, использовав LabelEncoder. С помощью cross_val_score (cv=10) оцените accuracy на каждом из этих наборов данных следующих алгоритмов:

<ul>
<li>DecisionTree, считающий все признаки вещественными</li>

<li>DecisionTree, считающий все признаки категориальными</li>

<li>DecisionTree, считающий все признаки вещественными + one-hot-encoding всех признаков</li>

<li>DecisionTreeClassifier из sklearn</li>
</ul>

Запишите результат в pd.DataFrame (по строкам - наборы данных, по столбцам - алгоритмы).
Рекомендации:

Чтобы cross_val_score вычисляла точность, нужно передать scorer=make_scorer(accuracy_score), обе фукнции из sklearn.metrics.
Если вам позволяет память (а она скорее всего позволяет), указывайте параметр sparse=False в OneHotEncoder (если вы, конечно, используете его). Иначе вам придется добиваться того, чтобы ваша реализация дерева умела работать с разреженными матрицами (что тоже, в целом, не очень сложно).

In [31]:
# Ваш код здесь:


### Задание 3. Композиция деревьев

#### 3.1 [3 балла]

Загрузите датасет [winequality-red.csv](https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv) в датафрейм. Последний столбец - целевая переменная (содержит классы).

С помощью cross_val_score с cv=5 оцените качество (accuracy) следующих классификаторов:
<ul>
<li>DecisionTreeClassifier</li>
<li>BaggingClassifier со 100 деревьями</li>
<li>BaggingClassifier со 100 деревьями; каждое дерево обучается только по половине случайно выбранных признаков (см. параметры метода)</li>
<li>RandomForestClassifier со 100 деревьями</li>
</ul>
Значение получается шумное, но в целом у вас должно получиться, что качество возрастает с каждым следующим алгоритмом. Этот пример демонстрирует, что RandomForest - это более сложный алгоритм, чем бэггинг и бэггинг со случайными подпространствами.

In [30]:
# Ваш код здесь:
